In [14]:
import pypsa
import numpy as np
import pandas as pd

"""
The network:

A central bus that connects a solar generator, a wind generator, a battery, and a load.

"""

# Create the timeseries index
index = pd.date_range("2011-01-01 00:00", "2011-12-31 21:00", freq="3h")

# Cost assumptions (example values, should be refined)
cost_per_mw_solar = 1000  # in $/MW
cost_per_mw_wind = 1200  # in $/MW
cost_per_mwh_storage = 500  # in $/MWh

# Initialize the network
network = pypsa.Network()
network.set_snapshots(index)

# Add the buses
network.add("Bus", "Main bus")

# Add load
load_profile = np.full(len(index), 300)  # 300 MW constant load for each interval
network.add("Load", "Desired load", bus="Main bus", p_set=load_profile)

# Add generation
solar_availability = np.random.rand(len(index))
wind_onshore_availability = np.random.rand(len(index))
wind_offshore_availability = np.random.rand(len(index))

network.add("Generator", "Solar park", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_solar, p_max_pu=solar_availability)

network.add("Generator", "Wind farm onshore", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_wind, p_max_pu=wind_onshore_availability)

network.add("Generator", "Wind farm offshore", bus="Main bus", p_nom_extendable=True, 
            capital_cost=cost_per_mw_wind, p_max_pu=wind_offshore_availability)

# Add storage
network.add("StorageUnit", "Battery", bus="Main bus", p_nom_extendable=True,
            capital_cost=cost_per_mwh_storage)

# Define the objective function (minimize cost)
network.optimize(network.snapshots, solver_name='cbc')

# Output results
print("Optimal size of Solar Park: {:.2f} MW".format(network.generators.p_nom_opt["Solar park"]))
print("Optimal size of Wind Farm (onshore): {:.2f} MW".format(network.generators.p_nom_opt["Wind farm onshore"]))
print("Optimal size of Wind Farm (offshore): {:.2f} MW".format(network.generators.p_nom_opt["Wind farm offshore"]))
print("Optimal size of Battery Storage: {:.2f} MWh".format(network.storage_units.p_nom_opt["Battery"]))

INFO:linopy.model: Solve problem using Cbc solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|█████████████████████████████████████████████████████| 6/6 [00:00<00:00, 142.05it/s]
INFO:linopy.io: Writing time: 0.31s
INFO:linopy.solvers:Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc -printingOptions all -import /tmp/linopy-problem-24ie33ya.lp -solve -solu /tmp/linopy-solve-yn7bj8ih.sol (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 23357 (-17527) rows, 17522 (-2) columns and 61311 (-17532) elements
Perturbing problem by 0.001% of 27460.573 - largest nonzero change 0.0046103431 ( 0.00012633922%) - largest zero change 0.0046092441
0  Obj 0 Primal inf 1111142.7 (2920)
341  Obj 30.731316 Primal inf 1028609.8 (2975)
682  Obj 55.908171 Primal inf 976923.32 (3008)
1023  Obj 77.867764 Primal inf 921914.02 (3031)
1364  Obj 99.022289 Primal inf 874832.67 (3060)
1705  Obj 119.22243 Primal 

Optimal size of Solar Park: 216.74 MW
Optimal size of Wind Farm (onshore): 245.67 MW
Optimal size of Wind Farm (offshore): 287.30 MW
Optimal size of Battery Storage: 467.66 MWh
